### Import Libraries

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

C:\Users\al_kosa\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\al_kosa\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\al_kosa\anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Highlight the detected shape of the template on the image

In [3]:
def highlight_shape(img, temp_img, position):
    result = img.copy()
    result[position[1]: position[1] + temp_img.shape[0],
           position[0]: position[0] + temp_img.shape[1]][temp_img != 0] = [255, 0, 0]
    return result

### Divide template into k segments

In [152]:
def get_segments(template, k):
    def neighbours(pix):
        x = pix[0]
        for y in [pix[1]-1, pix[1]+1]:
            yield x, y

        y = pix[1]
        for x in [pix[0]-1, pix[0]+1]:
            yield x, y

        for x in [pix[0]-1, pix[0]+1]:
            for y in [pix[1]-1, pix[1]+1]:
                yield x, y

    segment_length = int(np.ceil(template.sum() / 255 / k))
    xy_pixels = np.nonzero(template)
    pixels = list(zip(xy_pixels[0], xy_pixels[1]))
    
    segments = []
    segment = np.zeros_like(template)
    queue = []

    old_pixel = pixels[330]  # Will deal with this later
    segment[old_pixel[0], old_pixel[1]] = 255
    pixels.remove(old_pixel)
    
    for i in range(1, int(template.sum()/255)):
        for x, y in neighbours(old_pixel):
            if (x, y) in pixels and (x, y) not in queue:
                queue.append((x,y))

        pixel = queue.pop(0)
        segment[pixel[0], pixel[1]] = 255
        pixels.remove(pixel)
        old_pixel = pixel

        if (i+1) % segment_length == 0:
            segments.append(segment)
            segment = np.zeros_like(template)
    else:
        if segment.any():
            segments.append(segment)
            
    return segments

### Read template image and video

In [213]:
temp = cv2.imread('hand_template.bmp', 0)
height, width = temp.shape[::]
cap = cv2.VideoCapture('test2.wmv')
frame_exists, frame = cap.read()

In [214]:
n_segments = 20
n_max = 100
alpha = 0.5

In [215]:
segments = get_segments(temp, n_segments)

# for segment in segs:
#     cv2.imshow('seg', segment)
#     cv2.waitKey(0)

# cv2.destroyAllWindows()

### For each frame:
* Convert the frame to gray scale
* Apply Canny edge detection
* Apply distance transformation
* Get the chamfer matching by correlating the transformed frame with the template
* Get the location of the maximum matched value
* Highlight the shape at that location

In [216]:
while frame_exists:    
    cm = []
    max_ind = []
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 1)
    edges = cv2.Canny(gray, 50, 200)
    dist = cv2.distanceTransform(255 - edges, cv2.DIST_L2, 3)
    dist = dist.astype(np.uint8)
    
    for seg in segments:
        cm.append((cv2.matchTemplate(255 - dist, seg, cv2.TM_CCORR) / seg.sum()))
        ind = np.unravel_index((-1*cm[-1]).argsort(axis=None), cm[-1].shape)
        max_ind.append(np.array([ind[0][:n_max], ind[1][:n_max]]))
    
    cost_old = alpha * (1 - (cm[0][tuple(max_ind[0])] / 255)) * 10000
    idx_tracker = [[x] for x in range(n_max)]
    for i in range(1, n_segments):
        cost = cost_old[:, None] + alpha * (1 - (cm[i][tuple(max_ind[i])] / 255)) * 10000 + \
               (1 - alpha) * (np.linalg.norm(max_ind[i] - max_ind[i - 1], axis=0))
        idx = np.unravel_index((cost).argsort(axis=None), cost.shape)
        idx = (idx[0][:n_max], idx[1][:n_max])
        cost_old = cost[idx]
        idx_tracker1 = idx_tracker.copy()
        idx_tracker = [idx_tracker1[x] + [y] for x, y in list(zip(idx[0], idx[1]))]
    top_left_idx = idx_tracker[0]

    hl_img = frame
    for i in range(n_segments):
        hl_img = highlight_shape(hl_img, segments[i], (max_ind[i][1][top_left_idx[i]], max_ind[i][0][top_left_idx[i]]))
        
    cv2.imshow('dt', dist)
    cv2.imshow('Edges', edges)
    cv2.imshow('Tracking', hl_img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_exists, frame = cap.read()

cap.release()
cv2.destroyAllWindows()